In [19]:
import torch
import sys

# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('/om5/user/smadan/training_scaffold_own/res/')
from models.models import get_model
from loader.loader import get_loader
import random
import pickle

In [ ]:
sys.path.append('/om5/user/smadan/')

In [2]:
# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 30 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.

# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = '../data/' # to run with the sample dataset, just set to 'hymenoptera_data'

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 1 # set to 1 if want to run on gpu.

MODEL_ARCH = 'simple_cnn'

In [7]:
loader = get_loader('multi_attribute_loader_file_list_shapenet')

In [8]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm import tqdm

In [14]:
# DATASET_NAME = 'mnist_rotation_eight_by_nine'
DATASET_NAME = 'shapenet_hand_picked'
NUM_EPOCHS = 2
BATCH_SIZE = 10
# ARCH = 'RESNET1'
# In[36]:


image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

GPU = 1

# NUM_CLASSES = (10,10,10,10)
NUM_CLASSES = 11

In [15]:
# model = get_model(ARCH,NUM_CLASSES)
# loader_new = get_loader('multi_attribute_loader_file_list_mnist_rotation')
loader_new = get_loader('multi_attribute_loader_file_list_shapenet')
file_list_root = '/om5/user/smadan/dataset_lists_openmind'
#att_path = '/data/graphics/toyota-pytorch/biased_dataset_generalization/dataset_lists/merged_colors_binned_attribute.p'
att_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'
shuffles = {'train':True,'val':True,'test':False}

In [16]:
use_gpu = GPU_MODE
if use_gpu:
    torch.cuda.set_device(CUDA_DEVICE)

count=0

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# data_dir = DATA_DIR
# dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
#          for x in ['train', 'test']}
# dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=BATCH_SIZE,
#                                                shuffle=True, num_workers=25)
#                 for x in ['train', 'test']}
# dset_sizes = {x: len(dsets[x]) for x in ['train', 'test']}
# dset_classes = dsets['train'].classes

In [17]:
################ GET FROM USER CONFIG - TODO #####################
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['train','test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])


# In[37]:

# Define VAE Model